# Process single cell profiles

## Import libraries

In [1]:
import pathlib
import pprint

import pandas as pd

from pycytominer import annotate, normalize, feature_select

## Set paths and variables

In [2]:
# Path to dir with cleaned data from single-cell QC
converted_dir = pathlib.Path("./data/cleaned_profiles")

# output path for single-cell profiles
output_dir = pathlib.Path("./data/single_cell_profiles")
output_dir.mkdir(parents=True, exist_ok=True)

# Extract the plate names from the file name
plate_names = [file.stem.split("_")[0] for file in converted_dir.glob("*.parquet")]

# path for platemap directory
platemap_dir = pathlib.Path("../metadata/updated_platemaps/")

# operations to perform for feature selection
feature_select_ops = [
    "variance_threshold",
    "correlation_threshold",
    "blocklist",
    "drop_na_columns",
]

## Set dictionary with plates to process

In [3]:
# Load the barcode_platemap file
barcode_platemap_df = pd.read_csv(
    pathlib.Path(f"{platemap_dir}/updated_barcode_platemap.csv").resolve()
)

# Create plate info dictionary
plate_info_dictionary = {
    name: {
        "profile_path": (converted_dir / f"{name}_cleaned.parquet").resolve(
            strict=True
        ),
        "platemap_path": (
            platemap_dir
            / f"{barcode_platemap_df.loc[barcode_platemap_df['plate_barcode'] == name, 'platemap_file'].values[0]}.csv"
        ).resolve(strict=True),
    }
    for name in plate_names
}

# View the dictionary to assess that all info is added correctly
pprint.pprint(plate_info_dictionary, indent=4)

{   'localhost240926150001': {   'platemap_path': PosixPath('/home/erikserrano/Development/targeted_fibrosis_drug_screen/metadata/updated_platemaps/Target_Selective_Library_Screen_Plate_1_with_pathways.csv'),
                                 'profile_path': PosixPath('/home/erikserrano/DataRepo/CFReT_screen_data/cleaned_profiles/localhost240926150001_cleaned.parquet')},
    'localhost240927060001': {   'platemap_path': PosixPath('/home/erikserrano/Development/targeted_fibrosis_drug_screen/metadata/updated_platemaps/Target_Selective_Library_Screen_Plate_1_with_pathways.csv'),
                                 'profile_path': PosixPath('/home/erikserrano/DataRepo/CFReT_screen_data/cleaned_profiles/localhost240927060001_cleaned.parquet')},
    'localhost240927120001': {   'platemap_path': PosixPath('/home/erikserrano/Development/targeted_fibrosis_drug_screen/metadata/updated_platemaps/Target_Selective_Library_Screen_Plate_1_with_pathways.csv'),
                                 'profile_pat

## Process data with pycytominer

In [4]:
for plate, info in plate_info_dictionary.items():
    print(f"Performing pycytominer pipeline for {plate}")
    output_annotated_file = str(
        pathlib.Path(f"{output_dir}/{plate}_sc_annotated.parquet")
    )
    output_normalized_file = str(
        pathlib.Path(f"{output_dir}/{plate}_sc_normalized.parquet")
    )
    output_feature_select_file = str(
        pathlib.Path(f"{output_dir}/{plate}_sc_feature_selected.parquet")
    )

    profile_df = pd.read_parquet(info["profile_path"])
    platemap_df = pd.read_csv(info["platemap_path"])

    print("Performing annotation for", plate, "...")
    # Step 1: Annotation
    annotate(
        profiles=profile_df,
        platemap=platemap_df,
        join_on=["Metadata_well_position", "Image_Metadata_Well"],
        output_file=output_annotated_file,
        output_type="parquet",
    )

    # Load the annotated parquet file to fix metadata columns names
    annotated_df = pd.read_parquet(output_annotated_file)

    # Rename columns using the rename() function
    column_name_mapping = {
        "Image_Metadata_Site": "Metadata_Site",
    }

    annotated_df.rename(columns=column_name_mapping, inplace=True)

    # Save the modified DataFrame back to the same location
    annotated_df.to_parquet(output_annotated_file, index=False)

    # Step 2: Normalization
    normalized_df = normalize(
        profiles=output_annotated_file,
        method="standardize",
        output_file=output_normalized_file,
        output_type="parquet",
    )

    print("Performing feature selection for", plate, "...")
    # Step 3: Feature selection
    feature_select(
        output_normalized_file,
        operation=feature_select_ops,
        na_cutoff=0,
        output_file=output_feature_select_file,
        output_type="parquet",
    )
    print(
        f"Annotation, normalization, and feature selection have been performed for {plate}"
    )

Performing pycytominer pipeline for localhost240928120001
Performing annotation for localhost240928120001 ...
Performing feature selection for localhost240928120001 ...
Annotation, normalization, and feature selection have been performed for localhost240928120001
Performing pycytominer pipeline for localhost240926150001
Performing annotation for localhost240926150001 ...
Performing feature selection for localhost240926150001 ...
Annotation, normalization, and feature selection have been performed for localhost240926150001
Performing pycytominer pipeline for localhost240927060001
Performing annotation for localhost240927060001 ...
Performing feature selection for localhost240927060001 ...
Annotation, normalization, and feature selection have been performed for localhost240927060001
Performing pycytominer pipeline for localhost240927120001
Performing annotation for localhost240927120001 ...
Performing feature selection for localhost240927120001 ...
Annotation, normalization, and feature 

In [5]:
# Check output file
test_df = pd.read_parquet(output_feature_select_file)

print(test_df.shape)
test_df.head(2)

(12902, 684)


,Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_cell_type,Metadata_heart_failure_type,Metadata_treatment,Metadata_Pathway,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,...,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_02_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_03_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_00_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_01_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_02_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_03_256,Nuclei_Texture_SumEntropy_PM_3_01_256,Nuclei_Texture_SumVariance_ER_3_01_256,Nuclei_Texture_SumVariance_Mitochondria_3_03_256,Nuclei_Texture_SumVariance_PM_3_01_256
0,B,2,7,healthy,None,DMSO,None,907.921642,66.684968,848.242523,...,-0.592359,-0.893847,0.327355,0.489503,0.326130,0.012874,0.543994,-0.270493,-0.051838,-0.138941
1,B,2,7,healthy,None,DMSO,None,396.033681,82.690457,348.336337,...,-0.191731,-0.387954,-0.795001,-0.903883,-0.944581,-0.987366,0.824843,-0.370290,-0.442164,0.320484
